In [318]:
import networkx as nx
import plotly
import plotly.graph_objects as go
import json
import matplotlib.pyplot as plt

In [319]:
with open("data/events.json") as f:
    data = json.load(f)

In [320]:
len(data)

29

In [333]:
nodes = []
nodes_pos = []
last_y_value = 0
node_dict = dict()

for i in range(len(data)):
    idx = data[i]["id"]
    y_value = 0
    
    if data[i]["tag"]=="real":
        y_value=1
    
    if data[i]["tag"]=="end":
        y_value = 1.0/int(data[i]["id"])
    """
    if data[i]["date"]<-10000000:
        data[i]["date"] = -10000000+100000*last_y_value
        last_y_value=last_y_value+1
        y_value = last_y_value
    
    if data[i]["date"]>10000000:
        data[i]["date"] = 10000000-100000*last_y_value
        last_y_value=last_y_value+1
        y_value = last_y_value
    """
    
    position = (data[i]["date"], y_value)
    extra_data = data[i]
    extra_data["pos"] = position
    
    nodes.append((idx, extra_data))
    nodes_pos.append(position)
    node_dict[data[i]["id"]] = data[i]
        


edges = []
nodes_ids = [x[0] for x in nodes]
for i in range(len(data)):
    idx = data[i]["id"]
    if "answers" in data[i].keys():
        for r in data[i]["answers"][0]["responses"]:
            if "next_event_id" in r.keys() :
                next_id = r["next_event_id"]
                if next_id not in nodes_ids:
                    continue
                color = "green"
                if r["type"]=="bad":
                    color="red"
                edges.append((idx,next_id,{"color":color}))

In [334]:
G = nx.Graph()

In [335]:
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [336]:
G.edges

EdgeView([(-8, 2), (-6, 8), (-5, 11), (-4, 13), (-3, 7), (-2, 1), (-2, 3), (-1, 4), (-1, 5), (-1, 6), (-1, 10), (-1, 11), (0, 13), (1, 5), (1, 4), (2, 6), (3, 4), (5, 7), (6, 8), (7, 9), (8, 10), (10, 11), (13, 15)])

In [337]:
G.nodes

NodeView((-8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20))

In [338]:
G.edges

EdgeView([(-8, 2), (-6, 8), (-5, 11), (-4, 13), (-3, 7), (-2, 1), (-2, 3), (-1, 4), (-1, 5), (-1, 6), (-1, 10), (-1, 11), (0, 13), (1, 5), (1, 4), (2, 6), (3, 4), (5, 7), (6, 8), (7, 9), (8, 10), (10, 11), (13, 15)])

In [339]:
nodes_pos

[(4000000000, -0.125),
 (4000000000, -0.14285714285714285),
 (4000000000, -0.16666666666666666),
 (4000000000, -0.2),
 (4000000000, -0.25),
 (4000000000, -0.3333333333333333),
 (4000000000, -0.5),
 (4000000000, -1.0),
 (-10000000000, 1),
 (-776, 1),
 (1963, 1),
 (-8000, 1),
 (-3000, 1),
 (-753, 1),
 (1986, 1),
 (-44, 1),
 (2001, 1),
 (20, 1),
 (2020, 1),
 (2030, 1),
 (79, 1),
 (-4000000000, 1),
 (1460, 1),
 (-380000000, 1),
 (-200000000, 1),
 (-65000000, 1),
 (-10000, 1),
 (476, 1),
 (1492, 1)]

In [340]:
def get_color(tag):
    if tag is None:
        return "green"
    elif tag=="fake":
        return "red"
    elif tag=="end":
        return "black"
    return "green"

In [341]:
edge_x = []
edge_y = []
for edge in G.edges():
    print(edge)
    if "pos" in G.nodes[edge[0]].keys() and "pos" in G.nodes[edge[1]].keys():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_y.append(y0)
        edge_y.append(y1)

print(edge_x)
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

(-8, 2)
(-6, 8)
(-5, 11)
(-4, 13)
(-3, 7)
(-2, 1)
(-2, 3)
(-1, 4)
(-1, 5)
(-1, 6)
(-1, 10)
(-1, 11)
(0, 13)
(1, 5)
(1, 4)
(2, 6)
(3, 4)
(5, 7)
(6, 8)
(7, 9)
(8, 10)
(10, 11)
(13, 15)
[4000000000, 1963, 4000000000, 2001, 4000000000, 2030, 4000000000, -4000000000, 4000000000, -44, 4000000000, -776, 4000000000, -8000, 4000000000, -3000, 4000000000, -753, 4000000000, 1986, 4000000000, 2020, 4000000000, 2030, -10000000000, -4000000000, -776, -753, -776, -3000, 1963, 1986, -8000, -3000, -753, -44, 1986, 2001, -44, 20, 2001, 2020, 2020, 2030, -4000000000, -380000000]


In [367]:
previous_events = dict([(node,[]) for node in G.nodes()])
following_events = dict([(node,[]) for node in G.nodes()])

for node in G.nodes():
    previous_ids = node_dict[node]["previous_ids"]
    next_ids = node_dict[node]["next_ids"]
    
    next_events[node] = [node_dict[i]["event_title"] for i in next_ids]
    previous_events[node] = [node_dict[i]["event_title"] for i in previous_ids]

In [370]:
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

nodes_ids = []
node_x = []
node_y = []
events = []
colors = []
for node in G.nodes():
    if "pos" in G.nodes[node].keys():
        n = G.nodes[node]
        x, y = n['pos']
        events.append(n["event_title"])
        colors.append(get_color(n["tag"]))
        nodes_ids = n["id"]
        node_x.append(x)
        node_y.append(y)
        
customdata=[(previous_events[node],next_events[node]) for node in G.nodes()]      

node_trace = go.Scatter(
    x=node_x, y=node_y, text=events,customdata=customdata,
    hovertemplate =
    '<br><b>Date</b>: %{x}<br>'+
    '<b>%{text}</b><br>'+'<b>In:</b>%{customdata[0]}<br>'+'<b>Out:</b>%{customdata[1]}<br>',
    mode='markers',
    hoverinfo='text',
    marker=dict(
        color=colors,
        size=10,
        line_width=2)
)

fig = go.Figure(data=[node_trace, edge_trace])
fig.show()

In [331]:
trace = go.Scatter(
    x=[1,2,3], y=[1,2,2], customdata=[(0,1) for x in range(3)],
    hovertemplate ='%{customdata[0]}',
    mode='markers',
    hoverinfo='text',
    marker=dict(
        color=colors,
        size=10,
        line_width=2)
)

fig = go.Figure(data=[trace])
fig.show()